In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os


In [2]:
#read csv file
data =pd.read_csv('movies.csv')



In [3]:
#display the shape of data 
data.shape

(4803, 24)

In [4]:
#check for null values
data.isna().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [5]:
#select relevent features for recommendation
selected_features = ['genres','keywords','tagline','cast','director']


In [6]:
data[selected_features].head()

,genres,keywords,tagline,cast,director
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton


In [7]:
#check for null values in selected attributes
data[selected_features].isna().sum()

genres       28
keywords    412
tagline     844
cast         43
director     30
dtype: int64

In [8]:
#Replacing the null values with null string
for feature in selected_features:
  data[feature] = data[feature].fillna('')
display (data.head())


,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [9]:
#combining all selected features
combined_features = data['genres']+' '+data['keywords']+' '+data['tagline']+' '+data['cast']+' '+data['director']
display (combined_features)


0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [10]:
#Converting the text data to feature vectors
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)
display (feature_vectors.shape)
display (feature_vectors.shape)
print (feature_vectors)


(4803, 17318)

(4803, 17318)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

In [11]:
#Getting the similarity scores using cosine similarity
similarity = cosine_similarity(feature_vectors)
print(similarity.shape)
print  (similarity )


(4803, 4803)
[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [12]:
#Enter the movie name to get Similarity 
movie_name = input(' Enter your favourite movie name : ')


 Enter your favourite movie name : superman


In [13]:
#Creating a list with all the movie names given in the dataset
movie_title_list = data['title'].tolist()
len(movie_title_list)


4803

In [14]:
#find movies which are matching to given movie name
find_close_match = difflib.get_close_matches(movie_name, movie_title_list)



In [15]:
print(find_close_match)

['Superman', 'Superman II', 'Superman III']


In [16]:
#we choose first movie from list of matching movies because it is the the most matching movie name  to the given movie name
close_match = find_close_match[0]

index_of_the_movie = data[data.title == close_match]['index'].values[0]
print(index_of_the_movie)


813


In [17]:
#displaying the cosine similarity score of the close_matched movie with other movies
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)


[(0, 0.06605757327786128), (1, 0.05494299286841542), (2, 0.022474655275199845), (3, 0.18229768118495995), (4, 0.04826852550842614), (5, 0.031153020937531885), (6, 0.007816453213423849), (7, 0.03428924698332946), (8, 0.020070942434549842), (9, 0.10380582477831074), (10, 0.20133128757695565), (11, 0.01951398074561013), (12, 0.025827307032424), (13, 0.016928223234223393), (14, 0.3188610536407353), (15, 0.018553049692010756), (16, 0.03251349500357992), (17, 0.02470681749845619), (18, 0.029643169623505952), (19, 0.04417369541584225), (20, 0.029947941232879607), (21, 0.011401180198063948), (22, 0.03639562527544031), (23, 0.025066323882186783), (24, 0.03654732000312345), (25, 0.0), (26, 0.03538948489610589), (27, 0.03529828509945782), (28, 0.038753351499167796), (29, 0.0142136220030666), (30, 0.025493376585254864), (31, 0.04277585721181915), (32, 0.03875437083506119), (33, 0.03407619761223045), (34, 0.0), (35, 0.035771481945846714), (36, 0.03223295986231689), (37, 0.020432362116956287), (38, 

In [18]:
#we sort the coine similarity  in decreasing order
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 



In [19]:
#we display the top 10 similar movie to selected movie(any number of movies can be displayed)
i=1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = data[data.index==index]['title'].values[0]
  if (i<11):
    print(i, ')',title_from_index)
    i+=1


1 ) Superman
2 ) Superman II
3 ) Superman IV: The Quest for Peace
4 ) Man of Steel
5 ) Superman III
6 ) Crimson Tide
7 ) Superman Returns
8 ) Batman Returns
9 ) Suicide Squad
10 ) The Killer Inside Me
